# **1412. Find the Quiet Students in All Exams**
``` console
Table: Student

+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| student_id          | int     |
| student_name        | varchar |
+---------------------+---------+
student_id is the primary key (column with unique values) for this table.
student_name is the name of the student.
 

Table: Exam

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| exam_id       | int     |
| student_id    | int     |
| score         | int     |
+---------------+---------+
(exam_id, student_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that the student with student_id had a score points in the exam with id exam_id.
 

A quiet student is the one who took at least one exam and did not score the highest or the lowest score.

Write a solution to report the students (student_id, student_name) being quiet in all exams. Do not return the student who has never taken any exam.

Return the result table ordered by student_id.

The result format is in the following example.

 

Example 1:

Input: 
Student table:
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 1           | Daniel        |
| 2           | Jade          |
| 3           | Stella        |
| 4           | Jonathan      |
| 5           | Will          |
+-------------+---------------+

Exam table:
+------------+--------------+-----------+
| exam_id    | student_id   | score     |
+------------+--------------+-----------+
| 10         |     1        |    70     |
| 10         |     2        |    80     |
| 10         |     3        |    90     |
| 20         |     1        |    80     |
| 30         |     1        |    70     |
| 30         |     3        |    80     |
| 30         |     4        |    90     |
| 40         |     1        |    60     |
| 40         |     2        |    70     |
| 40         |     4        |    80     |
+------------+--------------+-----------+

Output: 
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 2           | Jade          |
+-------------+---------------+

Explanation: 
For exam 1: Student 1 and 3 hold the lowest and high scores respectively.
For exam 2: Student 1 hold both highest and lowest score.
For exam 3 and 4: Studnet 1 and 4 hold the lowest and high scores respectively.
Student 2 and 5 have never got the highest or lowest in any of the exams.
Since student 5 is not taking any exam, he is excluded from the result.
So, we only return the information of Student 2.
```

**Solution-1:**

``` sql
# Write your MySQL query statement below
WITH cte AS (
SELECT e.student_id,e.exam_id,
rank() over(PARTITION BY e.exam_id ORDER BY e.score DESC) AS desc_rn,
rank() over(PARTITION BY e.exam_id ORDER BY e.score ASC) AS asc_rn  
FROM Exam e 
),
cte2 AS(
    SELECT student_id
    FROM cte
    WHERE student_id NOT IN(SELECT student_id FROM cte WHERE desc_rn = 1 OR asc_rn = 1)
)

SELECT DISTINCT s.student_id, s.student_name
FROM Student s
INNER JOIN cte2 e
ON s.student_id = e.student_id 
 

**Solution-2:**

``` sql
# Write your MySQL query statement below
with cte as
(
select student_id
from
Exam where (exam_id, score) in
(
    select exam_id, max(score) from Exam
    group by exam_id
)
union
select student_id
from
Exam where (exam_id, score) in
(
    select exam_id, min(score) from Exam
    group by exam_id
)
)
select student_id, student_name
from Student
where student_id not in (select student_id from cte)
and student_id in (select distinct(student_id) from Exam)


**Solution-3:**

``` sql
WITH cte AS(
    SELECT exam_id, exam.student_id, student_name, score, RANK() OVER(PARTITION BY exam_id ORDER BY score) rk1, RANK() OVER(PARTITION BY exam_id ORDER BY score DESC) rk2 
    FROM exam LEFT JOIN student
    ON exam.student_id = student.student_id
)

SELECT DISTINCT student_id, student_name
FROM cte
WHERE student_id NOT IN (SELECT student_id FROM cte WHERE rk1 = 1 or rk2 = 1)
ORDER BY student_id